# NBR 5422:2024 - teste metodologia em Python/ Jupyter
## Parâmetros iniciais

In [151]:
from normaslt import types, nbr5422
from tabulate import tabulate
from IPython.display import HTML, display
import pandas as pd
import importlib

# Recarrega as libs para fins de desenvolvimento
importlib.reload(types)
importlib.reload(nbr5422);

In [161]:
# Dados de projeto
Us = types.tensao._362kV
rugTerreno = types.rug.B
alt = 100
dra = 0.9
h = 11
regiao = types.regiao.SE
hmax = 0.
L = 450.
hmed = 25.
dcond = 2e-2
VrB = 15.
# Gap inicial
d = 4.
gaptorre = types.gap.CONDUTOR_BRACO
alpha = 0.33

## Cálculo dos parâmetros de projeto

In [165]:
# 4.8.5 - Fator de correção atmosférico
# Deve-se entrar a distancia no ar antes de calcular os gaps?
kafl = nbr5422.fatAtmFrenteLenta(dra, h, d)
# 4.9.4.3 - Fator de conversão do período de integração
kint = nbr5422.fatCorrInt3s(rugTerreno)
# 4.10.3.7 - Fator de correcao de rugosidade
Vr = nbr5422.fatCorrRug(rugTerreno) * VrB
# 4.10.3.9 - Fator de turbulencia;
Vp10 = Vr * nbr5422.fatTurb(regiao)
# 8.2.2.1
Vp30s = nbr5422.fatCorrInt30s(rugTerreno) * Vp10 
# 8.2.3.4
Vp3s = 1.15 * kint * Vp10
# 7.2.1.2. Parcelas de segurança
Pstip = 0.90
Pslim = 0.60
# 7.2.5. Parcelas elétricas
if 'Kcstip' not in locals():
    Kcstip = 1.51

if 'Kcslim' not in locals():
      Kcslim = 1.40

# Projeto de isolamento
zfl = 0.06
if 'Fsfl' not in locals():
    if Us.nom() <= 69:
        Fsfl = 3
    elif Us.nom() <= 440:
        Fsfl = 2.5
    elif Us.nom() <= 525:
        Fsfl = 2.3
    else:
        Fsfl = 2.2

# Pode-se definir o fator Kg na torre ou usar o valor típico pelo tipo do gap
if 'kgtorre' not in locals():
    kgtorre = nbr5422.fatKgFTFrenteLenta(gaptorre)

kgvao = nbr5422.fatKgFFFrenteLenta(types.gap.CONDUTOR_PARALELO, alpha)

# Ação do vento
# Temperatura de referência para pressão de vento
t = 15
mu = nbr5422.massaAr(alt, t)

# 8.4.2. Pressão dinâmica de referência - 10 min e 3 s
q010 = 0.5 * mu * Vp10^2
q03 = 0.5 * mu * Vp3^2
Cxc = nbr5422.arrastoCabos(d)
Gc = nbr5422.fatorVentoCabo(rugTerreno, h)
Gl = nbr5422.fatorVentoVao(L)
# 8.5.2.2. Ação do vento sobre os cabos
Ac = q010 * Cxc * Gc * Gl * d * L * sin(omega)^2
fprintf('Ação do vento sobre cabo - vento 10 minutos:\n')
fprintf('- Coeficiente de arrasto (Cxc):  %d\n', Cxc)
fprintf('- Fator combinado de vento (Gc): %d\n', Gc)
fprintf('- Fator de efetividade (Gl):     %d\n', Gl)
fprintf('- Pressão dinâmica (q0):         %d Pa\n', q010)

# 8.8.3.1. Força exercida pelo vento, suporte cilindrico
Atc = q010 * cxtc(d, Vp) * Gt * dtc  * Ltc * (sin(theta))^3

# Ângulo de balanço
ang = nbr5422.anguloBalanco(v, q0, d, pcond, Vv, Vp)

# Projeção horizontal

# Faixa de passagem

TypeError: unsupported operand type(s) for ^: 'float' and 'int'

# Distâncias de segurança
Cria uma tabela com as alturas do suporte para cada obstáculo e flecha

In [140]:
flecha = {10., 11., 12., 13} # a ser calculado
obsValidos = {types.obs.PEDESTRE, types.obs.MAQ_AGRICOLA, types.obs.RODOVIA, types.obs.FERROVIA_NAO_ELETRIFICADA, types.obs.FERROVIA_ELETRIFICADA}
Fsfl = 2.2
data = []
for obs in obsValidos:
    row_data = []
    for reg in types.amp:
        row_data.append(nbr5422.distSegurancaVert(obs, reg, 0., Us.nom(), Fsfl, kafl))

    data.append(row_data)

table = tabulate(data, headers=['Dv'+e.suf() for e in types.amp], showindex=[e.value for e in obsValidos], tablefmt='html')
display(HTML(table))

,"Dvtip,n","Dvtip,s","Dvlim,n","Dvlim,s"
acesso somente a pedestres,7.13679,6.30596,6.23679,5.63679
máquinas agrícolas,7.56004,6.76018,6.66004,6.06004
ferrovia eletrificada,13.1796,12.3545,12.2796,11.6796
ferrovia não eletrificada,10.06,9.26018,9.16004,8.56004
rodovia,9.06004,8.26018,8.16004,7.56004
